# Κρυπταριθμητικά παζλ

Αντικατάσταση των χαρακτήρων με ψηφία έτσι ώστε να προκύπτει έγκυρο αποτέλεσμα.

```
      CP
+     IS
+    FUN
--------
=   TRUE
```

μια έγκυρη ανάθεση είναι

```
      23
+     74
+    968
--------
=   1065
```

https://developers.google.com/optimization/cp/cryptarithmetic

In [1]:
# Copyright 2010-2011 Google
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Cryptarithmetic puzzle

First attempt to solve equation CP + IS + FUN = TRUE
where each letter represents a unique digit.

This problem has 72 different solutions in base 10.
"""

from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from os import abort

def CPIsFun():
  # Constraint programming engine
  solver = pywrapcp.Solver('CP is fun!');
  
  kBase = 10
  
  # Decision variables.
  digits = list(range(0, kBase))
  digits_without_zero = list(range(1, kBase))
  c = solver.IntVar(digits_without_zero, 'C');
  p = solver.IntVar(digits, 'P');
  i = solver.IntVar(digits_without_zero, 'I');
  s = solver.IntVar(digits, 'S');
  f = solver.IntVar(digits_without_zero, 'F');
  u = solver.IntVar(digits, 'U');
  n = solver.IntVar(digits, 'N');
  t = solver.IntVar(digits_without_zero, 'T');
  r = solver.IntVar(digits, 'R');
  e = solver.IntVar(digits, 'E');

  # We need to group variables in a list to use the constraint AllDifferent.
  letters = [c, p, i, s, f, u, n, t, r, e]
  
  # Verify that we have enough digits.
  assert kBase >= len(letters)

  # Define constraints.
  solver.Add(solver.AllDifferent(letters))
  
  # CP + IS + FUN = TRUE
  solver.Add (p + s + n + kBase * (c + i + u) + kBase * kBase * f ==
              e + kBase * u + kBase * kBase * r + kBase * kBase * kBase * t)

  db = solver.Phase(letters, solver.INT_VAR_DEFAULT,
                             solver.INT_VALUE_DEFAULT)
  solver.NewSearch(db)
  
  while solver.NextSolution():
    print(letters)
    # Is CP + IS + FUN = TRUE?
    assert (kBase*c.Value() +  p.Value() + kBase*i.Value() + s.Value() +
            kBase*kBase*f.Value() + kBase*u.Value() + n.Value() == 
            kBase*kBase*kBase*t.Value() + kBase*kBase*r.Value() + 
            kBase*u.Value() + e.Value()) 
  
  solver.EndSearch()

  return


if __name__ == '__main__':
  CPIsFun()

[C(2), P(3), I(7), S(4), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(3), I(7), S(5), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(5), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(6), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(8), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(8), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(3), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(8), F(9), U(6), N(3), T(1), R(0), E(5)]
[C(2), P(5), I(7), S(3), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(5), I(7), S(3), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(6), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(8), F(9), U(4), N(3), T(1), R(0), E(6)]
[C(2), P(6), I(7), S(3), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(6), I(7), S(5), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(8), I(7), S(3), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(8), I(7), S(3), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(8), I(7), S(4),

In [2]:
# Copyright 2010-2011 Google
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Cryptarithmetic puzzle.

First attempt to solve equation CP + IS + FUN = TRUE
where each letter represents a unique digit.

This problem has 72 different solutions in base 10.
"""


from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count


def CPIsFunSat():
    """Solve the CP+IS+FUN==TRUE cryptarithm."""
    # Constraint programming engine
    model = cp_model.CpModel()

    base = 10

    c = model.NewIntVar(1, base - 1, 'C')
    p = model.NewIntVar(0, base - 1, 'P')
    i = model.NewIntVar(1, base - 1, 'I')
    s = model.NewIntVar(0, base - 1, 'S')
    f = model.NewIntVar(1, base - 1, 'F')
    u = model.NewIntVar(0, base - 1, 'U')
    n = model.NewIntVar(0, base - 1, 'N')
    t = model.NewIntVar(1, base - 1, 'T')
    r = model.NewIntVar(0, base - 1, 'R')
    e = model.NewIntVar(0, base - 1, 'E')

    # We need to group variables in a list to use the constraint AllDifferent.
    letters = [c, p, i, s, f, u, n, t, r, e]

    # Verify that we have enough digits.
    assert base >= len(letters)

    # Define constraints.
    model.AddAllDifferent(letters)

    # CP + IS + FUN = TRUE
    model.Add(c * base + p + i * base + s + f * base * base + u * base +
              n == t * base * base * base + r * base * base + u * base + e)

    ### Solve model.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(letters)
    status = solver.SearchForAllSolutions(model, solution_printer)

    print()
    print('Statistics')
    print('  - status          : %s' % solver.StatusName(status))
    print('  - conflicts       : %i' % solver.NumConflicts())
    print('  - branches        : %i' % solver.NumBranches())
    print('  - wall time       : %f s' % solver.WallTime())
    print('  - solutions found : %i' % solution_printer.solution_count())


if __name__ == '__main__':
    CPIsFunSat()

C=6 P=4 I=3 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=8 F=9 U=2 N=5 T=1 R=0 E=7 
C=3 P=2 I=6 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=3 P=2 I=6 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=2 P=3 I=7 S=6 F=9 U=8 N=5 T=1 R=0 E=4 
C=2 P=3 I=7 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=2 P=3 I=7 S=5 F=9 U=4 N=8 T=1 R=0 E=6 
C=2 P=3 I=7 S=8 F=9 U=4 N=5 T=1 R=0 E=6 
C=2 P=3 I=7 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=2 P=3 I=7 S=5 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=3 I=2 S=5 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=3 I=2 S=6 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=2 I=3 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=2 I=3 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=7 P=3 I=2 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=7 P=3 I=2 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=7 P=4 I=2 S=8 F=9 U=6 N=3 T=1 R=0 E=5 
C=7 P=8 I=2 S=4 F=9 U=6 N=3 T=1 R=0 E=5 
C=7 P=8 I=2 S=3 F=9 U=6 N=4 T=1 R=0 E=5 
C=7 P=4 I=2 S=3 F=9 U=6 N=8 T=1 R=0 E=5 
C=7 P=6 I=2 S=3 F=9 U=8 N=5 T=1 R=0 E=4 
C=7 P=5 I=2 S=3 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=8 I=2 S=3 F=9 U=4 N=5 T=1 R=0 E=6 
C=7 P=8 I=2 S=5 